<a href="https://colab.research.google.com/github/CristinaMarsh/Reproduce/blob/main/Uncertaintyforecasting/normalizing_flow_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/honglinwen/Conditional-normalizing-flow-for-wind-power-forecasting.git

Cloning into 'Conditional-normalizing-flow-for-wind-power-forecasting'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), 752.32 KiB | 4.20 MiB/s, done.


In [2]:

!pip install UMNN
!pip install properscoring
!pip install git+https://github.com/bayesiains/nflows.git
!pip install torch-summary
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for UMNN: filename=UMNN-1.68-py3-none-any.whl size=36282 sha256=b22d86c0ce28ec2deaf462d04568ec0d2b3cd200dc9aa35eea8929028c212fc0
  Stored in directory: /root/.cache/pip/wheels/1d/ee/7e/95a477fd53650135d7dfac61e01b4f0e6f18e2a507dd5202da
Successfully built UMNN
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/bayesiains/nflows.git to /tmp/pip-req-build-h_wru7fi
  Running command git clone --filter=blob:none --quiet https://github.com/bayesiains/nflows.git /tmp/pip-req-build-h_wru7fi
  Encountered 2 file(s) that should have been pointers, but weren't:
        examples/conditional_moons.ipynb
        examples/moons.ipynb
  Resolved https://github.com/bayesiains/

In [3]:
import torch
from torch import nn
from torch import optim

from nflows.flows.base import Flow
from nflows.distributions.normal import ConditionalDiagonalNormal, StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform, MaskedPiecewiseQuadraticAutoregressiveTransform, MaskedPiecewiseLinearAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.autoregressive import MaskedUMNNAutoregressiveTransform, MaskedPiecewiseRationalQuadraticAutoregressiveTransform
#from nflows.transforms.nonlinearities import Logit
from nflows.nn.nets import ResidualNet
from nflows.transforms.base import (
    CompositeTransform,
    InputOutsideDomain,
    InverseTransform,
    Transform,
)


from nflows.transforms.autoregressive import AutoregressiveTransform
from nflows.transforms import made as made_module
from nflows.utils import torchutils
from torch.nn import functional as F

In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:

import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pandas as pd
import numpy as np
import copy

In [6]:
def fill_ndarray(t1):
    for i in range(t1.shape[1]):  
        temp_col = t1[:, i]  
        nan_num = np.count_nonzero(temp_col != temp_col)
        if nan_num != 0:  
            temp_not_nan_col = temp_col[temp_col == temp_col]  
            temp_col[np.isnan(temp_col)] = temp_not_nan_col.mean()  
    return t1

def get_data(df):
  data = df['TARGETVAR'].values.reshape((-1,1))
  return data

def get_nwp(d):
  cls = d.columns
  data = []
  for i in range(4):
    data.append(d[cls[i+3]].values.reshape((-1,1)))
  data = np.hstack(data)
  return data
     

In [7]:

data = pd.read_csv('/content/Conditional-normalizing-flow-for-wind-power-forecasting/Task15_W_Zone1.csv',delimiter=',')

power = get_data(data); x = np.float32(get_nwp(data))
y = np.float32(fill_ndarray(power))

In [24]:
power.shape

(16800, 1)

In [23]:
x.shape

torch.Size([16800, 4])

In [8]:
y = torch.tensor(y).float()
x = torch.tensor(x).float()

In [9]:

import math

index = math.floor(x.shape[0]*0.8)

y_train = y[:index]
x_train = x[:index,:]

y_test = y[index:]
x_test = x[index:,:]


if torch.cuda.is_available():
  x_train, y_train, x_test, y_test = x_train.cuda(), y_train.cuda(), x_test.cuda(), y_test.cuda()

In [10]:

print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

torch.Size([13440, 4]) torch.Size([13440, 1]) torch.Size([3360, 4]) torch.Size([3360, 1])


In [11]:


class Sigmoid(Transform):
    def __init__(self, temperature=1, eps=1e-6, learn_temperature=False):
        super().__init__()
        self.eps = eps
        if learn_temperature:
            self.temperature = nn.Parameter(torch.Tensor([temperature]))
        else:
            self.temperature = torch.Tensor([temperature]).to(device)

    def forward(self, inputs, context=None):
        inputs = self.temperature * inputs
        outputs = torch.sigmoid(inputs)
        logabsdet = torchutils.sum_except_batch(
            torch.log(self.temperature) - F.softplus(-inputs) - F.softplus(inputs)
        )
        return outputs, logabsdet

    def inverse(self, inputs, context=None):
        if torch.min(inputs) < 0 or torch.max(inputs) > 1:
            raise InputOutsideDomain()

        inputs = torch.clamp(inputs, self.eps, 1 - self.eps).to(device)

        outputs = (1 / self.temperature) * (torch.log(inputs) - torch.log1p(-inputs))
        logabsdet = -torchutils.sum_except_batch(
            torch.log(self.temperature)
            - F.softplus(-self.temperature * outputs)
            - F.softplus(self.temperature * outputs)
        )
        return outputs, logabsdet


class Logit(InverseTransform):
    def __init__(self, temperature=1, eps=1e-6):
        super().__init__(Sigmoid(temperature=temperature, eps=eps))

     

In [12]:

from nflows.transforms.base import InputOutsideDomain
from nflows.utils import torchutils


def unconstrained_linear_spline(
    inputs, unnormalized_pdf, inverse=False, tail_bound=1.0, tails="linear"
):
    inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
    outside_interval_mask = ~inside_interval_mask

    outputs = torch.zeros_like(inputs)
    logabsdet = torch.zeros_like(inputs)

    if tails == "linear":
        outputs[outside_interval_mask] = inputs[outside_interval_mask]
        logabsdet[outside_interval_mask] = 0
    else:
        raise RuntimeError("{} tails are not implemented.".format(tails))

    if torch.any(inside_interval_mask):
        outputs[inside_interval_mask], logabsdet[inside_interval_mask] = linear_spline(
            inputs=inputs[inside_interval_mask],
            unnormalized_pdf=unnormalized_pdf[inside_interval_mask, :],
            inverse=inverse,
            left=-tail_bound,
            right=tail_bound,
            bottom=-tail_bound,
            top=tail_bound,
        )

    return outputs, logabsdet
     


In [13]:
def linear_spline(
    inputs, unnormalized_pdf, inverse=False, left=0.0, right=1.0, bottom=0.0, top=1.0
):
    """
    Reference:
    > Müller et al., Neural Importance Sampling, arXiv:1808.03856, 2018.
    """
    if torch.min(inputs) < left or torch.max(inputs) > right:
        raise InputOutsideDomain()

    if inverse:
        inputs = (inputs - bottom) / (top - bottom)
    else:
        inputs = (inputs - left) / (right - left)

    num_bins = unnormalized_pdf.size(-1)

    pdf = F.softmax(unnormalized_pdf, dim=-1)

    cdf = torch.cumsum(pdf, dim=-1)
    cdf[..., -1] = 1.0
    cdf = F.pad(cdf, pad=(1, 0), mode="constant", value=0.0)

    if inverse:
        inv_bin_idx = torchutils.searchsorted(cdf, inputs)

        bin_boundaries = (
            torch.linspace(0, 1, num_bins + 1)
            .view([1] * inputs.dim() + [-1])
            .expand(*inputs.shape, -1)
        ).to(device)

        slopes = (cdf[..., 1:] - cdf[..., :-1]) / (
            bin_boundaries[..., 1:] - bin_boundaries[..., :-1]
        )
        offsets = cdf[..., 1:] - slopes * bin_boundaries[..., 1:]

        inv_bin_idx = inv_bin_idx.unsqueeze(-1)
        input_slopes = slopes.gather(-1, inv_bin_idx)[..., 0]
        input_offsets = offsets.gather(-1, inv_bin_idx)[..., 0]

        outputs = (inputs - input_offsets) / input_slopes
        outputs = torch.clamp(outputs, 0, 1)

        logabsdet = -torch.log(input_slopes)
    else:
        bin_pos = inputs * num_bins

        bin_idx = torch.floor(bin_pos).long()
        bin_idx[bin_idx >= num_bins] = num_bins - 1

        alpha = bin_pos - bin_idx.float()

        input_pdfs = pdf.gather(-1, bin_idx[..., None])[..., 0]

        outputs = cdf.gather(-1, bin_idx[..., None])[..., 0]
        outputs += alpha * input_pdfs
        outputs = torch.clamp(outputs, 0, 1)

        bin_width = 1.0 / num_bins
        logabsdet = torch.log(input_pdfs) - np.log(bin_width)

    if inverse:
        outputs = outputs * (right - left) + left
    else:
        outputs = outputs * (top - bottom) + bottom

    return outputs, logabsdet

In [14]:
class MaskedUnconstrainedPiecewiseLinearAutoregressiveTransform(AutoregressiveTransform):
    def __init__(
        self,
        num_bins,
        features,
        hidden_features,
        context_features=None,
        num_blocks=2,
        use_residual_blocks=True,
        random_mask=False,
        activation=F.relu,
        dropout_probability=0.0,
        use_batch_norm=False,
        tail_bound=1.0,
        tail = 'linear',
    ):
        self.num_bins = num_bins
        self.features = features
        self.tail_bound = tail_bound
        self.tail = tail
        made = made_module.MADE(
            features=features,
            hidden_features=hidden_features,
            context_features=context_features,
            num_blocks=num_blocks,
            output_multiplier=self._output_dim_multiplier(),
            use_residual_blocks=use_residual_blocks,
            random_mask=random_mask,
            activation=activation,
            dropout_probability=dropout_probability,
            use_batch_norm=use_batch_norm,
        )
        super().__init__(made)

    def _output_dim_multiplier(self):
        return self.num_bins

    def _elementwise(self, inputs, autoregressive_params, inverse=False):
        batch_size = inputs.shape[0]

        unnormalized_pdf = autoregressive_params.view(
            batch_size, self.features, self._output_dim_multiplier()
        )
        
        outputs, logabsdet = unconstrained_linear_spline(
            inputs=inputs, unnormalized_pdf=unnormalized_pdf, inverse=inverse, tail_bound=self.tail_bound, tails=self.tail
        )

        return outputs, torchutils.sum_except_batch(logabsdet)

    def _elementwise_forward(self, inputs, autoregressive_params):
        return self._elementwise(inputs, autoregressive_params)

    def _elementwise_inverse(self, inputs, autoregressive_params):
        return self._elementwise(inputs, autoregressive_params, inverse=True)

In [15]:
from nflows.transforms.splines.cubic import unconstrained_cubic_spline

class MaskedUnconstrainedPiecewiseCubicAutoregressiveTransform(AutoregressiveTransform):
    def __init__(
        self,
        num_bins,
        features,
        hidden_features,
        context_features=None,
        num_blocks=2,
        use_residual_blocks=True,
        random_mask=False,
        activation=F.relu,
        dropout_probability=0.0,
        use_batch_norm=False,
    ):
        self.num_bins = num_bins
        self.features = features
        made = made_module.MADE(
            features=features,
            hidden_features=hidden_features,
            context_features=context_features,
            num_blocks=num_blocks,
            output_multiplier=self._output_dim_multiplier(),
            use_residual_blocks=use_residual_blocks,
            random_mask=random_mask,
            activation=activation,
            dropout_probability=dropout_probability,
            use_batch_norm=use_batch_norm,
        )
        super(MaskedUnconstrainedPiecewiseCubicAutoregressiveTransform, self).__init__(made)

    def _output_dim_multiplier(self):
        return self.num_bins * 2 + 2

    def _elementwise(self, inputs, autoregressive_params, inverse=False):
        batch_size = inputs.shape[0]

        transform_params = autoregressive_params.view(
            batch_size, self.features, self.num_bins * 2 + 2
        )

        unnormalized_widths = transform_params[..., : self.num_bins]
        unnormalized_heights = transform_params[..., self.num_bins : 2 * self.num_bins]
        derivatives = transform_params[..., 2 * self.num_bins :]
        unnorm_derivatives_left = derivatives[..., 0][..., None]
        unnorm_derivatives_right = derivatives[..., 1][..., None]

        if hasattr(self.autoregressive_net, "hidden_features"):
            unnormalized_widths /= np.sqrt(self.autoregressive_net.hidden_features)
            unnormalized_heights /= np.sqrt(self.autoregressive_net.hidden_features)

        outputs, logabsdet = unconstrained_cubic_spline(
            inputs=inputs,
            unnormalized_widths=unnormalized_widths,
            unnormalized_heights=unnormalized_heights,
            unnorm_derivatives_left=unnorm_derivatives_left,
            unnorm_derivatives_right=unnorm_derivatives_right,
            inverse=inverse,
            tail_bound=1.0,
            tails="linear",
        )
        return outputs, torchutils.sum_except_batch(logabsdet)

    def _elementwise_forward(self, inputs, autoregressive_params):
        return self._elementwise(inputs, autoregressive_params)

    def _elementwise_inverse(self, inputs, autoregressive_params):
        return self._elementwise(inputs, autoregressive_params, inverse=True)
     

In [16]:
model = nn.Sequential(
          nn.Linear(4, 512),
          nn.ReLU(),
          nn.Linear(512, 512),
          nn.ReLU(),
          nn.Linear(512, 2)
        )
     

In [17]:
num_layers = 5
base_dist = ConditionalDiagonalNormal(shape=[1],context_encoder=model)
#base_dist = StandardNormal(shape=[1])

transforms = []

#transforms.append(Logit(eps=1e-3))
#logit needs a cuda version
for _ in range(num_layers):
    transforms.append(ReversePermutation(features=1))
    #transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(features=1,hidden_features=64,context_features=4,num_bins=10,tails = "linear"))
    #transforms.append(MaskedPiecewiseQuadraticAutoregressiveTransform(features=1,hidden_features=256,context_features=4,num_bins=10,tails = "linear"))
    #transforms.append(MaskedAffineAutoregressiveTransform(features=1,hidden_features=256,context_features=4))
    #transforms.append(MaskedUMNNAutoregressiveTransform(features=1,hidden_features=512,context_features=4,integrand_net_layers=[256, 256, 256]))
    #transforms.append(MaskedUnconstrainedPiecewiseLinearAutoregressiveTransform(features=1,hidden_features=256,context_features=4,num_bins=10,tail_bound=1.0,tail = 'linear'))
    transforms.append(MaskedUnconstrainedPiecewiseCubicAutoregressiveTransform(features=1,hidden_features=256,context_features=4,num_bins=10))

transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist)
if torch.cuda.is_available():
  flow = flow.cuda()

#optimizer = optim.Adam(flow.parameters(),lr=1e-3)
optimizer = optim.Adam(flow.parameters(),lr=1e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[300,600], gamma=0.3)

In [18]:
num_iter = 1000
for i in range(num_iter):
    optimizer.zero_grad()
    loss = -flow.log_prob(inputs=y_train, context=x_train).mean()
    if i%10 == 0:
      print('iteration',i,':',loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

iteration 0 : 4.480475902557373
iteration 10 : 0.481362521648407
iteration 20 : -0.20111586153507233
iteration 30 : -0.39565396308898926
iteration 40 : -0.5149353742599487
iteration 50 : -0.4903756380081177
iteration 60 : -0.563126802444458
iteration 70 : -0.6017073392868042
iteration 80 : -0.6897176504135132
iteration 90 : -0.7085854411125183
iteration 100 : -0.6676489114761353
iteration 110 : -0.7365836501121521
iteration 120 : -0.743061363697052
iteration 130 : -0.6964942812919617
iteration 140 : -0.689651370048523
iteration 150 : -0.7512606978416443
iteration 160 : -0.7919875383377075
iteration 170 : -0.44511619210243225
iteration 180 : -0.6521691083908081
iteration 190 : -0.6865114569664001
iteration 200 : -0.7506049275398254
iteration 210 : -0.7681019902229309
iteration 220 : -0.7359240055084229
iteration 230 : -0.7307777404785156
iteration 240 : -0.6892122030258179
iteration 250 : -0.7821463942527771
iteration 260 : -0.8055863380432129
iteration 270 : -0.7495648860931396
iterati

In [19]:

import properscoring as ps
y_true = y_test.cpu().numpy()

In [20]:

score = 0
samples = []
with torch.no_grad():
  for i in range(y_true.shape[0]):
    sample = flow.sample(1000,context=x_test[i,:].reshape((-1,4))).cpu().numpy()
    sample = sample.squeeze()
    score += ps.crps_ensemble(y_true[i,0] , sample)
    samples.append(list(sample))
samples = np.array(samples)
print(samples.shape)
print(score/y_true.shape[0])

(3360, 1000)
0.09139957400376703
